### Comparativo de retorno entre SELIC e IBOV

In [110]:
import yfinance as yf
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import date, datetime

In [109]:
def consulta_bc(codigo_bcb, data_ini = '01-01-1900', data_fim = date.today().strftime('%d/%m/%Y')):
  url = f'http://api.bcb.gov.br/dados/serie/bcdata.sgs.{codigo_bcb}/dados?formato=json&dataInicial={data_ini}&dataFinal={data_fim}'
  df = pd.read_json(url)
  df['data'] = pd.to_datetime(df['data'], dayfirst=True)
  df.set_index('data', inplace=True)
  return df

In [99]:
start_date = '2003-01-01'

In [101]:
ibov = yf.download('^BVSP', start=start_date)['Close']
ibov

[*********************100%***********************]  1 of 1 completed


Date
2003-01-02     11603.0
2003-01-03     11600.0
2003-01-06     12020.0
2003-01-07     11876.0
2003-01-08     11786.0
                ...   
2023-08-31    115742.0
2023-09-01    117893.0
2023-09-04    117777.0
2023-09-05    117331.0
2023-09-06    115985.0
Name: Close, Length: 5122, dtype: float64

In [102]:
ibov_acum = ibov/ibov.iloc[0]
ibov_acum

Date
2003-01-02     1.000000
2003-01-03     0.999741
2003-01-06     1.035939
2003-01-07     1.023528
2003-01-08     1.015772
                ...    
2023-08-31     9.975179
2023-09-01    10.160562
2023-09-04    10.150565
2023-09-05    10.112126
2023-09-06     9.996122
Name: Close, Length: 5122, dtype: float64

In [113]:
fig_ibov = px.line(ibov_acum, x=ibov_acum.index, y=ibov_acum, title='Retorno IBOV')

fig_ibov.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig_ibov.show()

In [100]:
data_inicial = datetime.strptime(start_date, '%Y-%m-%d').strftime('%d/%m/%Y')
data_inicial


'01/01/2003'

In [104]:
cdi = consulta_bc(12, data_inicial)
cdi

,valor
data,
2002-12-31,0.088048
2003-01-02,0.088016
2003-01-03,0.087984
2003-01-06,0.088048
2003-01-07,0.088048
...,...
2023-08-31,0.049037
2023-09-01,0.049037
2023-09-04,0.049037


In [105]:
selic = consulta_bc(11, data_inicial)
selic

,valor
data,
2002-12-31,0.088270
2003-01-02,0.088270
2003-01-03,0.088270
2003-01-06,0.088270
2003-01-07,0.088270
...,...
2023-08-31,0.049037
2023-09-01,0.049037
2023-09-04,0.049037


In [106]:
cdi_acum = (1 + cdi[cdi.index >= start_date]/100).cumprod()
cdi_acum.iloc[0] = 1
cdi_acum

,valor
data,
2003-01-02,1.000000
2003-01-03,1.001761
2003-01-06,1.002643
2003-01-07,1.003526
2003-01-08,1.004409
...,...
2023-08-31,9.038187
2023-09-01,9.042619
2023-09-04,9.047053


In [116]:
selic_acum = (1 + selic[selic.index >= start_date]/100).cumprod()
selic_acum.iloc[0] = 1
selic_acum

,valor
data,
2003-01-02,1.000000
2003-01-03,1.001766
2003-01-06,1.002650
2003-01-07,1.003535
2003-01-08,1.004421
...,...
2023-08-31,9.114703
2023-09-01,9.119172
2023-09-04,9.123644


In [118]:
fig_selic = px.line(selic_acum, x=selic_acum.index, y=selic_acum.valor, title='Retorno SELIC')

fig_selic.update_xaxes(
    rangeslider_visible=True,
    rangeselector=dict(
        buttons=list([
            dict(count=1, label="1m", step="month", stepmode="backward"),
            dict(count=6, label="6m", step="month", stepmode="backward"),
            dict(count=1, label="YTD", step="year", stepmode="todate"),
            dict(count=1, label="1y", step="year", stepmode="backward"),
            dict(step="all")
        ])
    )
)
fig_selic.show()

In [119]:
fig = make_subplots(rows=1, cols=1)
fig.add_trace(go.Scatter(name='IBOV', x=ibov_acum.index, y=ibov_acum), row=1, col=1)
fig.add_trace(go.Scatter(name='SELIC', x=selic_acum.index, y=selic_acum.valor), row=1, col=1)
fig.update_layout(title='SELIC vs IBOV', template='plotly_dark')
fig.show()